In [1]:
import numpy as np
import pandas as pd

In [2]:
# livestock
Beef = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Beef')
Milk = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Milk')
Calves = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Calves')
Hogs = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Hogs')
Broilers = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Broilers')
Layers = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardSubMaster.xlsx', sheet_name='Layers')

In [3]:
# List of Tennessee counties
counties = [
    "Anderson", "Bedford", "Benton", "Bledsoe", "Blount", "Bradley", "Campbell", 
    "Cannon", "Carroll", "Carter", "Cheatham", "Chester", "Claiborne", "Clay", 
    "Cocke", "Coffee", "Crockett", "Cumberland", "Davidson", "De Kalb", "Decatur", 
    "Dickson", "Dyer", "Fayette", "Fentress", "Franklin", "Gibson", "Giles", 
    "Grainger", "Greene", "Grundy", "Hamblen", "Hamilton", "Hancock", "Hardeman", 
    "Hardin", "Hawkins", "Haywood", "Henderson", "Henry", "Hickman", "Houston", 
    "Humphreys", "Jackson", "Jefferson", "Johnson", "Knox", "Lake", "Lauderdale", 
    "Lawrence", "Lewis", "Lincoln", "Loudon", "Macon", "Madison", "Marion", "Marshall", 
    "Maury", "Mcminn", "Mcnairy", "Meigs", "Monroe", "Montgomery", "Moore", "Morgan", 
    "Obion", "Overton", "Perry", "Pickett", "Polk", "Putnam", "Rhea", "Roane", 
    "Robertson", "Rutherford", "Scott", "Sequatchie", "Sevier", "Shelby", "Smith", 
    "Stewart", "Sullivan", "Sumner", "Tipton", "Trousdale", "Unicoi", "Union", 
    "Van Buren", "Warren", "Washington", "Wayne", "Weakley", "White", "Williamson", "Wilson"
]

# Years range
years = list(range(2014, 2024))  # From 2014 to 2023

# Create DataFrame using a list comprehension for Cartesian product of counties and years
df = pd.DataFrame([(county, year) for county in counties for year in years], columns=['County', 'Year'])

In [4]:
# List of livestock data and their respective column names in the main DataFrame
livestock_dfs = [
    (Beef, 'Beef'),
    (Milk, 'Milk'),
    (Calves, 'Calves'),
    (Hogs, 'Hogs'),
    (Broilers, 'Broilers'),
    (Layers, 'Layers')
]

# Reshape each livestock DataFrame and merge it into the main DataFrame
for livestock_df, column_name in livestock_dfs:
    long_df = pd.melt(livestock_df, id_vars=['County'], var_name='Year', value_name=column_name)
    long_df['Year'] = pd.to_numeric(long_df['Year'])
    df = pd.merge(df, long_df, on=['County', 'Year'], how='left')

# Save the final DataFrame to a CSV file
# df.to_csv('Livestock.csv', index=False)

In [12]:
import numpy as np
import pandas as pd

Corn = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardMaster.xlsx', sheet_name='Corn')
Soy = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardMaster.xlsx', sheet_name='Soy')
Livestock = pd.read_excel('C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardMaster.xlsx', sheet_name='Livestock')

In [18]:
# step 1
# change data types to numeric for Corn['Planted','Harvested','Yield','Production'],
# Soy['Planted','Harvested','Yield','Production'], Livestock['Beef','Milk','Calves','Hogs','Broilers','Layers']

# step 2
# for each 'County' for each of the 3 dfs, determine the mean for each feature above, rounded to 1 decimal,
# impute 0 if all nulls

#step 3
# for each null space, fill in the mean; add the name of the imputed feature to the 'Estimated Values' column
# ex: 'Planted, Yield, Production'

# step 4
# write to 3 sheets on one Excel file

In [22]:
import pandas as pd
import numpy as np

# Function to detect and replace null values, spaces, or strange unicode characters
def detect_nulls_and_strange_chars(df):
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    df.replace(r'[^\x00-\x7F]+', np.nan, regex=True, inplace=True)
    return df

# Function to fill NaNs and track imputed features, leaving nulls blank for Corn and Soy
def fill_na_and_track(df, columns, fill_zeros=False):
    df['Estimated Values'] = ''
    for col in columns:
        mean_values = df.groupby('County')[col].transform(lambda x: round(x.mean(), 1) if x.notnull().any() else (0 if fill_zeros else np.nan))
        df['Estimated Values'] = df.apply(lambda row: row['Estimated Values'] + (col + ', ') if pd.isnull(row[col]) else row['Estimated Values'], axis=1)
        df[col].fillna(mean_values, inplace=True)
    df['Estimated Values'] = df['Estimated Values'].str.rstrip(', ')
    return df

# Load the original file with potential null values or strange characters
file_path = 'C:/Users/benja/Documents/Corn_Soy_Dashboard/DashboardMaster.xlsx'
Corn_original = pd.read_excel(file_path, sheet_name='Corn')
Soy_original = pd.read_excel(file_path, sheet_name='Soy')
Livestock_original = pd.read_excel(file_path, sheet_name='Livestock')

# Detect null values, spaces, or strange unicode characters
Corn_checked = detect_nulls_and_strange_chars(Corn_original)
Soy_checked = detect_nulls_and_strange_chars(Soy_original)
Livestock_checked = detect_nulls_and_strange_chars(Livestock_original)

# Apply the function to all DataFrames
Corn_filled = fill_na_and_track(Corn_checked, corn_columns, fill_zeros=False)
Soy_filled = fill_na_and_track(Soy_checked, soy_columns, fill_zeros=False)
Livestock_filled = fill_na_and_track(Livestock_checked, livestock_columns, fill_zeros=True)

# Save the results to a new Excel file with multiple sheets
output_path_final = 'C:/Users/benja/Documents/Corn_Soy_Dashboard/Filled_Dashboard_Final2.xlsx'
with pd.ExcelWriter(output_path_final) as writer:
    Corn_filled.to_excel(writer, sheet_name='Corn_Filled', index=False)
    Soy_filled.to_excel(writer, sheet_name='Soy_Filled', index=False)
    Livestock_filled.to_excel(writer, sheet_name='Livestock_Filled', index=False)

output_path_final

'C:/Users/benja/Documents/Corn_Soy_Dashboard/Filled_Dashboard_Final2.xlsx'